# The Hired Hand

**Apprentissage Automatique pour la Prédiction de Changement d'Emploi**

---

## Table des Matières

1. [Description du Projet et du Jeu de Données](#1-description-du-projet-et-du-jeu-de-données)
   - [1.1 Objectif du Projet](#11-objectif-du-projet)
   - [1.2 Solutions Existantes](#12-solutions-existantes)
   - [1.3 Informations sur le Jeu de Données](#13-informations-sur-le-jeu-de-données)
   - [1.4 Hypothèses Initiales](#14-hypothèses-initiales)
2. [Importation des Bibliothèques](#2-importation-des-bibliothèques)
3. [Accès aux Données](#3-accès-aux-données)
4. [Analyse Exploratoire du Jeu de Données](#4-analyse-exploratoire-du-jeu-de-données)
   - [4.1 Analyse des Métadonnées](#41-analyse-des-métadonnées)
   - [4.2 Analyse des Valeurs Manquantes](#42-analyse-des-valeurs-manquantes)
   - [4.3 Distributions et Valeurs Aberrantes](#43-distributions-et-valeurs-aberrantes)
   - [4.4 Étude de la Variable Cible](#44-étude-de-la-variable-cible)
   - [4.5 Corrélations](#45-corrélations)
   - [4.6 Clustering Non Supervisé](#46-clustering-non-supervisé)
   - [4.7 Feature Engineering](#47-feature-engineering)
   - [4.8 Synthèse EDA et Vérification des Hypothèses](#48-synthèse-eda-et-vérification-des-hypothèses)
5. [Modélisation](#5-modélisation)
   - [5.1 Préparation des Données](#51-préparation-des-données)
   - [5.2 Modèles de Référence](#52-modèles-de-référence)
   - [5.3 Optimisation](#53-optimisation)
   - [5.4 Évaluation Finale](#54-évaluation-finale)
6. [Analyse Critique](#6-analyse-critique)
   - [6.1 Analyse des Erreurs](#61-analyse-des-erreurs)
   - [6.2 Feature Importance](#62-feature-importance)
   - [6.3 Retour sur les Hypothèses](#63-retour-sur-les-hypothèses)
7. [Conclusion](#7-conclusion)

---

## 1. Description du Projet et du Jeu de Données

### 1.1 Objectif du Projet

Ce projet applique des techniques d'Apprentissage Automatique pour prédire si un candidat cherche activement un nouvel emploi.

**Contexte métier :** Une entreprise de formation en Data Science souhaite identifier parmi ses participants ceux qui cherchent un emploi, afin de :
- Mieux cibler les candidats pour des offres d'emploi
- Comprendre les facteurs qui poussent à chercher un nouvel emploi
- Optimiser les ressources de placement

**Objectifs techniques :**
- Construire un modèle de classification binaire (cherche emploi vs ne cherche pas)
- Identifier les facteurs clés influençant la recherche d'emploi
- Appliquer une méthodologie ML rigoureuse

### 1.2 Solutions Existantes

**Approche Traditionnelle :**
Les départements RH utilisent des filtres manuels (expérience, diplôme, localisation) pour identifier les candidats. Cette approche est subjective et ne capture pas les interactions complexes entre facteurs.

**Solutions ML courantes :**
- **Modèles de référence :** Régression Logistique, KNN
- **Modèles ensemblistes :** Random Forest, XGBoost, Gradient Boosting

**Constats de la littérature :**
- Les méthodes ensemblistes surpassent généralement les modèles simples
- L'ingénierie des caractéristiques impacte significativement les performances
- La gestion du déséquilibre des classes est cruciale

### 1.3 Informations sur le Jeu de Données

**Source :** [HR Analytics: Job Change of Data Scientists](https://www.kaggle.com/datasets/arashnic/hr-analytics-job-change-of-data-scientists)

**Caractéristiques :**
- **Taille :** ~19 000 instances, 14 caractéristiques
- **Type :** Données tabulaires mixtes (numériques et catégorielles)
- **Cible :** `target` (1 = cherche un emploi, 0 = ne cherche pas)

**Variables principales :**
| Variable | Description |
|----------|-------------|
| `city_development_index` | Indice de développement de la ville |
| `experience` | Années d'expérience |
| `company_size` | Taille de l'entreprise actuelle |
| `company_type` | Type d'entreprise |
| `last_new_job` | Années depuis le dernier changement d'emploi |
| `training_hours` | Heures de formation suivies |
| `education_level` | Niveau d'éducation |
| `major_discipline` | Discipline principale |

### 1.4 Hypothèses Initiales

Avant d'explorer les données, voici nos hypothèses sur les facteurs qui influenceront la recherche d'emploi :

**H1 - Expérience professionnelle**
> Les candidats avec plus d'expérience seront moins enclins à chercher un nouvel emploi (stabilité de carrière).

**H2 - Niveau de développement de la ville**
> Les candidats dans des villes moins développées chercheront plus activement (moins d'opportunités locales).

**H3 - Situation professionnelle actuelle**
> Les candidats sans emploi actuel (`company_size` manquant) chercheront logiquement plus un emploi.

**H4 - Mobilité passée**
> Les candidats ayant changé d'emploi récemment (`last_new_job` faible) seront plus mobiles.

**H5 - Formation**
> Les candidats avec un niveau d'éducation élevé (Graduate, Masters) seront plus mobiles sur le marché.

---

➡️ **Ces hypothèses seront confrontées aux résultats en section 6.3**

---

## 2. Importation des Bibliothèques

In [ ]:
# Core
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ML
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

%matplotlib inline

# Configuration
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')

---

## 3. Accès aux Données

In [ ]:
DATA_URL = "https://raw.githubusercontent.com/Angry-Jay/ML_TheHiredHand/refs/heads/main/aug_train.csv"

df = pd.read_csv(DATA_URL)

print(f"Dataset chargé : {df.shape[0]} lignes, {df.shape[1]} colonnes")
df.head()

---

## 4. Analyse Exploratoire du Jeu de Données

### 4.1 Analyse des Métadonnées

Objectifs de cette section :
- Comprendre la structure du dataset (dimensions, types)
- Détecter les problèmes de qualité (doublons, colonnes inutiles)
- Vérifier l'absence de fuite de données

In [ ]:
# Structure du dataset
print(f"Dimensions : {df.shape[0]} lignes x {df.shape[1]} colonnes")
print(f"Doublons : {df.duplicated().sum()}")
print()
df.info()

In [ ]:
# Séparation par types
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"Numériques ({len(num_cols)}) : {num_cols}")
print(f"Catégorielles ({len(cat_cols)}) : {cat_cols}")

In [ ]:
# Statistiques descriptives (numériques)
df[num_cols].describe()

In [ ]:
# Cardinalité des variables catégorielles
print("Cardinalité des variables catégorielles :")
for col in cat_cols:
    print(f"  {col}: {df[col].nunique()} valeurs uniques")

In [ ]:
# Vérification de la fuite de données
suspect_keywords = ['salary', 'offer', 'hired', 'compensation']
leakage_cols = [col for col in df.columns if any(kw in col.lower() for kw in suspect_keywords)]

if leakage_cols:
    print(f"ATTENTION - Colonnes suspectes : {leakage_cols}")
else:
    print("Pas de fuite de données détectée.")

**Résumé 4.1 :**
- **19 158 instances**, 14 colonnes (13 prédicteurs + 1 cible)
- **Aucun doublon**
- **2 variables numériques** : `city_development_index`, `training_hours`
- **10 variables catégorielles** dont `city` (123 valeurs) = haute cardinalité
- `enrollee_id` = identifiant à exclure de la modélisation
- **Pas de fuite de données** détectée

### 4.2 Analyse des Valeurs Manquantes

In [ ]:
# Valeurs manquantes par colonne
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Manquants': missing,
    'Pourcentage': missing_pct
}).query('Manquants > 0').sort_values('Pourcentage', ascending=False)

print(f"Colonnes avec valeurs manquantes : {len(missing_df)} / {len(df.columns)}")
print(f"Lignes affectées : {df.isnull().any(axis=1).sum()} ({df.isnull().any(axis=1).sum()/len(df)*100:.1f}%)")
print()
missing_df

In [ ]:
# Visualisation des valeurs manquantes
fig, ax = plt.subplots(figsize=(10, 5))
bars = ax.barh(missing_df.index, missing_df['Pourcentage'], color='coral', edgecolor='black')
ax.set_xlabel('Pourcentage de valeurs manquantes (%)')
ax.set_title('Valeurs Manquantes par Variable', fontweight='bold')
ax.bar_label(bars, fmt='%.1f%%', padding=3)
ax.invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Co-occurrence des valeurs manquantes (insight clé)
both_missing = (df['company_size'].isnull() & df['company_type'].isnull()).sum()
print(f"company_size ET company_type manquantes ensemble : {both_missing} ({both_missing/len(df)*100:.1f}%)")

**Résumé 4.2 :**
- **8 colonnes** sur 14 ont des valeurs manquantes
- **53% des lignes** sont affectées (10 203 / 19 158)
- Les plus impactées : `company_type` (32%), `company_size` (31%), `gender` (24%)
- **Pattern clé** : `company_size` + `company_type` manquantes ensemble → candidats sans emploi actuel
- **Stratégie** : créer une catégorie "Non employé" plutôt qu'imputer (le manquant est informatif)

### 4.3 Distributions et Valeurs Aberrantes

In [ ]:
# Variables numériques à analyser (excluant enrollee_id et target)
num_features = ['city_development_index', 'training_hours']

# Histogrammes + Boxplots côte à côte
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

for idx, col in enumerate(num_features):
    # Histogramme
    axes[0, idx].hist(df[col], bins=30, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0, idx].set_title(f'Distribution de {col}', fontweight='bold')
    axes[0, idx].set_ylabel('Fréquence')
    
    # Boxplot
    axes[1, idx].boxplot(df[col].dropna(), vert=True)
    axes[1, idx].set_title(f'Boxplot de {col}', fontweight='bold')
    axes[1, idx].set_ylabel('Valeur')

plt.tight_layout()
plt.show()

In [ ]:
# Détection des outliers (méthode IQR)
print("Outliers détectés (méthode IQR) :")
for col in num_features:
    Q1, Q3 = df[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)]
    print(f"  {col}: {len(outliers)} outliers ({len(outliers)/len(df)*100:.1f}%)")

In [ ]:
# Distribution des principales variables catégorielles
cat_to_plot = ['education_level', 'experience', 'company_size', 'company_type', 'last_new_job', 'relevent_experience']

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.ravel()

for idx, col in enumerate(cat_to_plot):
    counts = df[col].value_counts()
    if len(counts) > 8:
        counts = counts.head(8)
    axes[idx].barh(counts.index.astype(str), counts.values, color='steelblue', edgecolor='black')
    axes[idx].set_title(col, fontweight='bold')
    axes[idx].invert_yaxis()

plt.tight_layout()
plt.show()

**Résumé 4.3 :**

**Variables numériques :**
- `city_development_index` : distribution asymétrique gauche (majorité villes développées ~0.9)
- `training_hours` : distribution asymétrique droite (mode ~50h, outliers jusqu'à 336h)
- **Outliers conservés** : ils représentent des comportements réels (engagement élevé, villes peu développées)

**Variables catégorielles - déséquilibres notables :**
- `education_level` : dominé par Graduate (60%)
- `company_type` : dominé par Pvt Ltd (83%)
- `relevent_experience` : 72% ont une expérience pertinente

### 4.4 Étude de la Variable Cible

In [ ]:
# Distribution de la variable cible
target_counts = df['target'].value_counts()
ratio = target_counts.max() / target_counts.min()

fig, ax = plt.subplots(figsize=(8, 4))
bars = ax.bar(['Ne cherche pas (0)', 'Cherche emploi (1)'], target_counts.values, 
              color=['#e74c3c', '#27ae60'], edgecolor='black')
ax.bar_label(bars, fmt='%d')
ax.set_title(f'Distribution de la Cible (Ratio {ratio:.1f}:1)', fontweight='bold')
ax.set_ylabel('Nombre')
plt.tight_layout()
plt.show()

print(f"Classe 0 (ne cherche pas) : {target_counts[0]} ({target_counts[0]/len(df)*100:.1f}%)")
print(f"Classe 1 (cherche emploi) : {target_counts[1]} ({target_counts[1]/len(df)*100:.1f}%)")
print(f"Ratio de déséquilibre : {ratio:.2f}:1")

In [ ]:
# Comparaison des distributions numériques par cible
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

for idx, col in enumerate(num_features):
    for target_val, color, label in [(0, '#e74c3c', 'Ne cherche pas'), (1, '#27ae60', 'Cherche')]:
        data = df[df['target'] == target_val][col]
        axes[idx].hist(data, bins=20, alpha=0.6, color=color, label=label, edgecolor='black')
    axes[idx].set_title(f'{col} par cible', fontweight='bold')
    axes[idx].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Taux de recherche d'emploi par variable catégorielle clé
key_cats = ['relevent_experience', 'enrolled_university', 'education_level', 'last_new_job']

fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()

for idx, col in enumerate(key_cats):
    rates = df.groupby(col)['target'].mean().sort_values(ascending=False) * 100
    axes[idx].barh(rates.index.astype(str), rates.values, color='steelblue', edgecolor='black')
    axes[idx].set_title(f'Taux de recherche par {col}', fontweight='bold')
    axes[idx].set_xlabel('%')
    axes[idx].invert_yaxis()

plt.tight_layout()
plt.show()

**Résumé 4.4 :**

**Déséquilibre des classes :** Ratio **3:1** (75% / 25%) → nécessite `class_weight='balanced'`

**Variables numériques :**
- `city_development_index` : **discriminant** - CDI faible → plus de chercheurs (confirme H2)
- `training_hours` : **peu discriminant** - distributions similaires entre classes

**Variables catégorielles - facteurs discriminants :**
| Variable | Observation |
|----------|-------------|
| `relevent_experience` | Sans exp. → 34% vs avec exp. → 22% |
| `enrolled_university` | Temps plein → 38%, non inscrit → 21% |
| `education_level` | Graduate → 28% (paradoxe : PhD → 14%) |
| `last_new_job` | "never" → 30%, >4 ans → 18% |

**Profil type du chercheur d'emploi :**
> Étudiant temps plein, niveau Graduate, sans expérience pertinente, premier emploi ou changement récent.

### 4.5 Corrélations

In [ ]:
# Corrélation des variables numériques (Pearson)
num_for_corr = ['city_development_index', 'training_hours', 'target']
corr_matrix = df[num_for_corr].corr()

plt.figure(figsize=(6, 5))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, square=True, fmt='.3f')
plt.title('Corrélation - Variables Numériques', fontweight='bold')
plt.tight_layout()
plt.show()

#### Association des variables catégorielles avec la cible (V de Cramér)

Le V de Cramér mesure la force d'association entre variables catégorielles :
- **< 0.1** : Négligeable | **0.1-0.3** : Faible | **> 0.3** : Modérée à forte

In [ ]:
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association

# Variables catégorielles à analyser (excluant identifiants)
categorical_cols = ['gender', 'relevent_experience', 'enrolled_university', 
                    'education_level', 'major_discipline', 'experience',
                    'company_size', 'company_type', 'last_new_job']

# Calcul du V de Cramér pour chaque variable catégorielle vs target
cramer_results = []
for col in categorical_cols:
    df_clean = df[[col, 'target']].dropna()
    contingency = pd.crosstab(df_clean[col], df_clean['target'])
    cramer_v = association(contingency, method='cramer')
    cramer_results.append({'Variable': col, 'V de Cramér': cramer_v})

cramer_df = pd.DataFrame(cramer_results).sort_values('V de Cramér', ascending=False)

# Visualisation
fig, ax = plt.subplots(figsize=(10, 5))
colors = ['#27ae60' if v > 0.1 else '#95a5a6' for v in cramer_df['V de Cramér']]
bars = ax.barh(cramer_df['Variable'], cramer_df['V de Cramér'], color=colors, edgecolor='black')
ax.axvline(x=0.1, color='red', linestyle='--', label='Seuil faible (0.1)')
ax.set_xlabel("V de Cramér")
ax.set_title("Association des Variables Catégorielles avec la Cible", fontweight='bold')
ax.bar_label(bars, fmt='%.3f', padding=3)
ax.legend()
ax.invert_yaxis()
plt.tight_layout()
plt.show()

**Résumé 4.5 :**

#### Analyse de la Matrice de Corrélation

- **Lien négatif fort avec le CDI (-0.342)** : L'indice de développement de la ville est la variable numérique la plus prédictive. Le coefficient négatif confirme que plus le CDI est élevé, moins les candidats cherchent à partir.

- **Indépendance des heures de formation (-0.022)** : La corrélation est proche de zéro, confirmant que le volume de formation n'aide pas à prédire l'intention de changement d'emploi.

- **Absence de multicolinéarité** : Quasi aucune corrélation entre CDI et training_hours (0.002) → ces variables apportent des informations distinctes.

#### Hiérarchie des Variables Catégorielles (V de Cramér)

**Prédicteurs clés (V > 0.1)** :
| Variable | V de Cramér | Observation |
|----------|-------------|-------------|
| `experience` | **0.192** | Variable catégorielle la plus déterminante |
| `enrolled_university` | **0.156** | Statut universitaire = indicateur majeur |
| `relevent_experience` | **0.128** | Complète le podium |

**Variables secondaires** : `education_level` (0.09) et `last_new_job` (0.08) sont juste sous le seuil mais restent plus informatives que `gender` ou `major_discipline`.

#### Synthèse Croisée

> **Le CDI est roi** : Avec r = -0.342, c'est le facteur numérique le plus puissant. Les candidats des villes moins développées sont les plus mobiles.
> 
> **Top 3 catégorielles** : `experience`, `enrolled_university`, `relevent_experience` - tous liés à la situation professionnelle actuelle.

### 4.6 Clustering Non Supervisé

**Objectif :** Vérifier si des patterns naturels émergent des données **sans utiliser la variable cible**.

Si le clustering identifie des groupes avec des taux de recherche différents, cela confirme que les features contiennent un signal prédictif exploitable.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


# Prétraitement pour clustering (sans target ni enrollee_id)
X_cluster = df.drop(['enrollee_id', 'target'], axis=1)
num_cols_cluster = X_cluster.select_dtypes(include=np.number).columns.tolist()
cat_cols_cluster = X_cluster.select_dtypes(include='object').columns.tolist()

cluster_preprocessor = ColumnTransformer([
    ('num', Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), num_cols_cluster),
    ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), cat_cols_cluster)
])

X_processed = cluster_preprocessor.fit_transform(X_cluster)
print(f"Données prétraitées : {X_processed.shape[1]} features")

In [ ]:
# Méthode du coude
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_processed)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(8, 4))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Nombre de clusters (k)')
plt.ylabel('Inertie')
plt.title('Méthode du Coude', fontweight='bold')
plt.xticks(K_range)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# K-Means avec k=4 (choisi d'après le coude)
k_optimal = 4
kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_processed)

# PCA pour visualisation
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_processed)

plt.figure(figsize=(10, 5))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis', alpha=0.5, s=10)
plt.colorbar(scatter, label='Cluster')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)')
plt.title(f'K-Means (k={k_optimal}) - Projection PCA', fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Variance expliquée par PCA : {pca.explained_variance_ratio_.sum()*100:.1f}%")

In [ ]:
# Analyse des clusters vs target
df_clusters = df.copy()
df_clusters['cluster'] = clusters

cluster_analysis = df_clusters.groupby('cluster')['target'].agg(['count', 'sum', 'mean'])
cluster_analysis.columns = ['Total', 'Chercheurs', 'Taux (%)']
cluster_analysis['Taux (%)'] = (cluster_analysis['Taux (%)'] * 100).round(1)
print("Distribution du target par cluster :")
print(cluster_analysis)

# Visualisation
fig, ax = plt.subplots(figsize=(8, 4))
colors = plt.cm.RdYlGn_r(cluster_analysis['Taux (%)'] / 100)
bars = ax.bar(cluster_analysis.index, cluster_analysis['Taux (%)'], color=colors, edgecolor='black')
ax.axhline(y=df['target'].mean()*100, color='red', linestyle='--', label=f'Moyenne globale ({df["target"].mean()*100:.1f}%)')
ax.set_xlabel('Cluster')
ax.set_ylabel('Taux de recherche (%)')
ax.set_title('Taux de Recherche d\'Emploi par Cluster', fontweight='bold')
ax.bar_label(bars, fmt='%.1f%%')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Profil caractéristique de chaque cluster
print("=== Profil des Clusters ===\n")

# Variables clés pour le profilage
profile_vars = {
    'city_development_index': 'CDI moyen',
    'training_hours': 'Heures formation (moy)',
}

# Calcul des profils
for cluster_id in sorted(df_clusters['cluster'].unique()):
    cluster_data = df_clusters[df_clusters['cluster'] == cluster_id]
    taux = cluster_data['target'].mean() * 100
    
    print(f"Cluster {cluster_id} (n={len(cluster_data)}, taux={taux:.1f}%)")
    print("-" * 40)
    
    # Variables numériques
    print(f"  CDI moyen : {cluster_data['city_development_index'].mean():.3f}")
    print(f"  % Employés : {cluster_data['company_size'].notna().mean()*100:.1f}%")
    print(f"  % Exp. pertinente : {(cluster_data['relevent_experience'] == 'Has relevent experience').mean()*100:.1f}%")
    
    # Top education level
    top_edu = cluster_data['education_level'].mode().iloc[0] if not cluster_data['education_level'].mode().empty else 'N/A'
    print(f"  Éducation dominante : {top_edu}")
    
    # Top enrolled_university
    top_univ = cluster_data['enrolled_university'].mode().iloc[0] if not cluster_data['enrolled_university'].mode().empty else 'N/A'
    print(f"  Inscription univ. dominante : {top_univ}")
    print()

**Résumé 4.6 :**

Le clustering K-Means (k=4) révèle **4 segments de candidats** avec des comportements distincts :

| Cluster | Taux | CDI | % Employés | % Exp. pert. | Interprétation |
|---------|------|-----|------------|--------------|----------------|
| 1 | **12.8%** | 0.899 | 82% | 100% | **Stables** : expérimentés, employés, CDI élevé |
| 0 | 17.9% | 0.857 | 72% | 77% | Intermédiaires |
| 2 | 27.2% | 0.893 | **39%** | **0.4%** | **Débutants** : peu employés, sans expérience |
| 3 | **49.0%** | **0.640** | 64% | 69% | **À risque** : CDI très faible |

**Lien avec les hypothèses :**
- **Cluster 3** (taux max 49%) : CDI = 0.640 → confirme **H2** (CDI faible = mobilité)
- **Cluster 2** (27%) : 39% employés, 0.4% exp. → confirme **H3** (sans emploi) et **H1** (sans exp.)
- **Cluster 1** (taux min 12.8%) : 100% exp. pertinente + 82% employés → profil stable (inverse de H1/H3)

**Conclusion clé :**
> Le clustering **sans utiliser le target** identifie des groupes corrélés au taux de recherche → les features contiennent des **patterns prédictifs naturels** qui valident nos hypothèses H1, H2, H3 avant même la modélisation supervisée.

### 4.7 Feature Engineering

Création de nouvelles variables pour améliorer la séparabilité des classes.

In [ ]:
# Feature : Statut d'emploi (basé sur le pattern de valeurs manquantes - section 4.2)
df['is_employed'] = df['company_size'].notna().astype(int)

# Vérification du pouvoir discriminant
employed_rate = df.groupby('is_employed')['target'].mean() * 100
print("Taux de recherche d'emploi :")
print(f"  Sans emploi (0) : {employed_rate[0]:.1f}%")
print(f"  Employé (1)     : {employed_rate[1]:.1f}%")

# V de Cramér pour la nouvelle feature
contingency = pd.crosstab(df['is_employed'], df['target'])
cramer_v = association(contingency, method='cramer')
print(f"\nV de Cramér (is_employed vs target) : {cramer_v:.3f}")

**Résumé 4.7 :**

**Feature créée : `is_employed`**
- Transforme le pattern de valeurs manquantes (section 4.2) en signal exploitable
- Sans emploi → **40.6%** cherchent vs Employé → **17.9%** cherchent
- **V de Cramér = 0.242** → Plus discriminante que toutes les autres variables catégorielles

**Pourquoi une seule feature ?** (Principe KISS)
> Les autres variables existent déjà sous forme exploitable. Seul le statut d'emploi capture une information nouvelle issue des valeurs manquantes.

✓ **Confirme H3** : Les candidats sans emploi cherchent logiquement plus un emploi.

### 4.8 Synthèse des Découvertes

**Données :** 19 158 instances, 15 colonnes (14 originales + `is_employed`)

**Variables numériques :**
- `city_development_index` : Distribution asymétrique gauche (majorité CDI > 0.9). Outliers conservés (villes peu développées = signal).
- `training_hours` : Distribution asymétrique droite (mode ~50h). 984 outliers (5.1%) conservés (engagement extrême = signal potentiel).

**Variables catégorielles - Déséquilibres notables :**
- `education_level` : Graduate (60%) | `company_type` : Pvt Ltd (83%) | `relevent_experience` : 72% ont une exp. pertinente

**Top 5 des variables discriminantes :**
| Rang | Variable | Mesure | Valeur |
|------|----------|--------|--------|
| 1 | `city_development_index` | Pearson | **-0.342** |
| 2 | `is_employed` | V Cramér | **0.242** |
| 3 | `experience` | V Cramér | 0.192 |
| 4 | `enrolled_university` | V Cramér | 0.156 |
| 5 | `relevent_experience` | V Cramér | 0.128 |

---

#### Vérification des Hypothèses

| Hypothèse | Statut | Justification EDA | Validation Clustering |
|-----------|--------|-------------------|----------------------|
| **H1** - Plus d'expérience → moins de recherche | ✅ Confirmée | V=0.192 | Cluster 1 : 100% exp. → 12.8% |
| **H2** - CDI faible → plus de recherche | ✅ **Confirmée** | r = -0.342 | Cluster 3 : CDI=0.64 → **49%** |
| **H3** - Sans emploi → plus de recherche | ✅ **Confirmée** | V=0.242 | Cluster 2 : 39% empl. → 27% |
| **H4** - Changement récent → plus mobile | ⚠️ Partielle | "never" → 30% | Non testé |
| **H5** - Éducation élevée → plus mobile | ❌ Infirmée | PhD → 14% | Éducation = Graduate partout |

**Double validation :** Les hypothèses H1, H2, H3 sont confirmées à la fois par l'analyse statistique (corrélations) ET par le clustering non supervisé — deux méthodes indépendantes.

---

#### Implications pour la Modélisation

1. **Déséquilibre 3:1** → utiliser `class_weight='balanced'`
2. **Valeurs manquantes** → stratégie par variable (catégorie "Unknown" ou imputation)
3. **Haute cardinalité** (`city` : 123 valeurs) → encodage target/fréquence
4. **Outliers conservés** → représentent des comportements réels

**Split 70/15/15 stratifié :**

| Ensemble | Proportion | Rôle |
|----------|------------|------|
| Train | 70% | Apprentissage |
| Validation | 15% | Ajustement hyperparamètres |
| Test | 15% | Évaluation finale |

**Prétraitement (Pipeline) :**
- Numériques → Imputation (médiane) + Standardisation
- Catégorielles → Imputation (mode) + One-Hot Encoding
- `class_weight='balanced'` → gère le déséquilibre 3:1

In [ ]:
# Préparation des données
X = df.drop(['enrollee_id', 'target'], axis=1)
y = df['target']

# Split 70/15/15 stratifié
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print(f"Train: {X_train.shape[0]} | Val: {X_val.shape[0]} | Test: {X_test.shape[0]}")

# Identification des types de features
num_features = X_train.select_dtypes(include=np.number).columns.tolist()
cat_features = X_train.select_dtypes(include='object').columns.tolist()

# Préprocesseur
preprocessor = ColumnTransformer([
    ('num', Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), num_features),
    ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), cat_features)
])

print(f"Features: {len(num_features)} numériques, {len(cat_features)} catégorielles")

### 5.2 Modèles de Référence

In [ ]:
# Comparaison avec/sans feature engineering
scoring = ['accuracy', 'f1_weighted', 'roc_auc']

# Version SANS is_employed
X_no_fe = X_train.drop('is_employed', axis=1)
num_feat_no_fe = X_no_fe.select_dtypes(include=np.number).columns.tolist()
cat_feat_no_fe = X_no_fe.select_dtypes(include='object').columns.tolist()

prep_no_fe = ColumnTransformer([
    ('num', Pipeline([('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), num_feat_no_fe),
    ('cat', Pipeline([('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), cat_feat_no_fe)
])

# Version AVEC is_employed (preprocessor déjà défini)

# Pipelines
pipelines = {
    'LR (sans FE)': Pipeline([('prep', prep_no_fe), ('model', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42))]),
    'LR (avec FE)': Pipeline([('prep', preprocessor), ('model', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42))]),
    'RF (sans FE)': Pipeline([('prep', prep_no_fe), ('model', RandomForestClassifier(class_weight='balanced', random_state=42))]),
    'RF (avec FE)': Pipeline([('prep', preprocessor), ('model', RandomForestClassifier(class_weight='balanced', random_state=42))])
}

# Cross-validation
results = {}
for name, pipeline in pipelines.items():
    X_data = X_no_fe if 'sans FE' in name else X_train
    scores = cross_val_score(pipeline, X_data, y_train, cv=5, scoring='roc_auc')
    results[name] = scores.mean()
    print(f"{name}: ROC-AUC = {scores.mean():.3f} (+/- {scores.std():.3f})")

# Gain du feature engineering
print(f"\nGain FE (Logistic Regression): +{(results['LR (avec FE)'] - results['LR (sans FE)'])*100:.1f}%")
print(f"Gain FE (Random Forest): +{(results['RF (avec FE)'] - results['RF (sans FE)'])*100:.1f}%")

### 5.3 Optimisation

**Stratégie :** Optimiser les deux meilleures familles de modèles

| Modèle | Justification |
|--------|---------------|
| **Logistic Regression** | Meilleur modèle de référence (0.788) → on l'optimise |
| **HistGradientBoosting** | Famille différente (boosting vs linéaire) → on explore |

Comparaison équitable des deux versions optimisées.

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

# 1. Optimisation Logistic Regression
lr_pipeline_opt = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42))
])

param_grid_lr = {
    'model__C': [0.01, 0.1, 1, 10],
    'model__solver': ['liblinear', 'lbfgs']
}

grid_lr = GridSearchCV(lr_pipeline_opt, param_grid_lr, cv=5, scoring='roc_auc', n_jobs=-1)
grid_lr.fit(X_train, y_train)

print("=== Logistic Regression ===")
print(f"Meilleurs paramètres : {grid_lr.best_params_}")
print(f"ROC-AUC (CV) : {grid_lr.best_score_:.3f}")

# 2. Optimisation HistGradientBoosting
hgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', HistGradientBoostingClassifier(class_weight='balanced', random_state=42))
])

param_grid_hgb = {
    'model__max_depth': [3, 5, 7],
    'model__learning_rate': [0.05, 0.1, 0.2],
    'model__max_iter': [100, 200]
}

grid_hgb = GridSearchCV(hgb_pipeline, param_grid_hgb, cv=5, scoring='roc_auc', n_jobs=-1)
grid_hgb.fit(X_train, y_train)

print("\n=== HistGradientBoosting ===")
print(f"Meilleurs paramètres : {grid_hgb.best_params_}")
print(f"ROC-AUC (CV) : {grid_hgb.best_score_:.3f}")

### 5.4 Évaluation Finale

In [ ]:
# Comparaison finale : modèles optimisés
models_final = {
    'LR (optimisé)': grid_lr.best_estimator_,
    'HGB (optimisé)': grid_hgb.best_estimator_
}

for name, model in models_final.items():
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    
    print(f"\n{'='*50}\n{name}\n{'='*50}")
    print(classification_report(y_test, y_pred))
    print(f"ROC-AUC: {roc_auc_score(y_test, y_proba):.3f}")

# Matrices de confusion
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
for idx, (name, model) in enumerate(models_final.items()):
    cm = confusion_matrix(y_test, model.predict(X_test))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx])
    axes[idx].set_title(name)
    axes[idx].set_xlabel('Prédit')
    axes[idx].set_ylabel('Réel')
plt.tight_layout()
plt.show()

**Résumé 5.4 :**

| Modèle | ROC-AUC | Rappel (classe 1) | Candidats détectés |
|--------|---------|-------------------|-------------------|
| LR (optimisé) | **0.797** | 0.77 | 551/716 (77%) |
| HGB (optimisé) | **0.797** | 0.78 | 560/716 (78%) |

**Observations :**
- Les deux modèles optimisés sont **pratiquement égaux** (ROC-AUC identique)
- HGB détecte 9 candidats de plus (+1 point de rappel) — différence marginale
- L'optimisation a apporté **+0.9%** vs LR de base (0.788 → 0.797)

**Choix du modèle final : Logistic Regression optimisée**

*Justification (principe KISS) :*
> Performances égales → choisir le modèle le plus simple et interprétable.
> 
> Ce résultat confirme que les relations sont **relativement linéaires** dans ce dataset, ce qui explique pourquoi LR surpassait déjà RF en référence.

---

## 6. Analyse Critique

### 6.1 Analyse des Erreurs

In [ ]:
# Modèle final : LR optimisé
best_model = grid_lr.best_estimator_
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# Analyse des erreurs
X_test_analysis = X_test.copy()
X_test_analysis['y_true'] = y_test.values
X_test_analysis['y_pred'] = y_pred
X_test_analysis['y_proba'] = y_proba

# Types d'erreurs
fn = X_test_analysis[(X_test_analysis['y_true'] == 1) & (X_test_analysis['y_pred'] == 0)]  # Faux Négatifs
fp = X_test_analysis[(X_test_analysis['y_true'] == 0) & (X_test_analysis['y_pred'] == 1)]  # Faux Positifs

print(f"Faux Négatifs (ratés) : {len(fn)} candidats chercheurs non détectés")
print(f"Faux Positifs : {len(fp)} candidats stables prédits comme chercheurs")

# Caractéristiques des faux négatifs (les plus coûteux pour le métier)
print("\n=== Profil des Faux Négatifs (candidats ratés) ===")
print(f"CDI moyen : {fn['city_development_index'].mean():.3f} (vs {X_test['city_development_index'].mean():.3f} global)")
print(f"% employés : {fn['is_employed'].mean()*100:.1f}% (vs {X_test['is_employed'].mean()*100:.1f}% global)")

**Résumé 6.1 :**

| Type d'erreur | Nombre | Interprétation |
|---------------|--------|----------------|
| Faux Négatifs | 165 | Candidats chercheurs **ratés** (23%) |
| Faux Positifs | 495 | Candidats stables mal classés |

**Profil des candidats ratés (FN) :**
- CDI moyen **plus élevé** (0.876 vs 0.827) → villes développées
- **80% employés** (vs 68.3%) → ont un emploi actuel

> Le modèle rate les candidats "atypiques" : employés dans des villes développées mais qui cherchent quand même. Ces cas contredisent le pattern principal appris (CDI bas + sans emploi → cherche).

### 6.2 Feature Importance

In [ ]:
# Feature importance via coefficients LR
lr_model = best_model.named_steps['model']
preprocessor_fitted = best_model.named_steps['preprocessor']

# Récupérer les noms des features après transformation
num_feat_names = num_features
cat_feat_names = preprocessor_fitted.named_transformers_['cat'].named_steps['encoder'].get_feature_names_out(cat_features).tolist()
all_feat_names = num_feat_names + cat_feat_names

# Coefficients
coef_df = pd.DataFrame({
    'Feature': all_feat_names,
    'Coefficient': lr_model.coef_[0]
}).sort_values('Coefficient', key=abs, ascending=False)

# Top 10 features les plus importantes
top_features = coef_df.head(10)

plt.figure(figsize=(10, 5))
colors = ['#27ae60' if c > 0 else '#e74c3c' for c in top_features['Coefficient']]
plt.barh(top_features['Feature'], top_features['Coefficient'], color=colors, edgecolor='black')
plt.xlabel('Coefficient (impact sur probabilité de chercher)')
plt.title('Top 10 Features - Logistic Regression', fontweight='bold')
plt.gca().invert_yaxis()
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.tight_layout()
plt.show()

print("Vert = augmente la probabilité de chercher | Rouge = diminue")

**Résumé 6.2 :**

**Top facteurs augmentant la probabilité de chercher (vert) :**
- `city_city_21`, `city_city_103`, `city_city_160`, `city_city_100` → certaines villes spécifiques

**Top facteurs diminuant la probabilité (rouge) :**
- `is_employed` → être employé réduit la recherche ✓ (confirme H3)
- `education_level_Primary School` → niveau primaire moins mobile
- `last_new_job_never` → premier emploi (relation complexe)
- `city_development_index` → CDI élevé réduit la recherche ✓ (confirme H2)

> Les coefficients du modèle confirment les insights de l'EDA : la **situation professionnelle** (`is_employed`, CDI) domine les facteurs démographiques.

### 6.3 Retour sur les Hypothèses

**Confrontation des hypothèses initiales (section 1.4) avec les résultats :**

| Hypothèse | Statut | Preuves EDA | Clustering | Modèle |
|-----------|--------|-------------|------------|--------|
| **H1** - Expérience → stabilité | ✅ Confirmée | V = 0.192 | Cluster 1 : 100% exp. → 12.8% | Coef. négatifs exp. élevée |
| **H2** - CDI faible → cherche | ✅ **Confirmée** | r = -0.342 | Cluster 3 : CDI=0.64 → **49%** | Top feature (coef. négatif) |
| **H3** - Sans emploi → cherche | ✅ **Confirmée** | V = 0.242 | Cluster 2 : 39% empl. → 27% | `is_employed` top coef. |
| **H4** - Changement récent → mobile | ⚠️ Partielle | "never" → 30% | — | Relation non linéaire |
| **H5** - Éducation élevée → mobile | ❌ Infirmée | PhD → 14% | Graduate partout | Coef. faible |

**Bilan : 3/5 hypothèses confirmées, 1 partielle, 1 infirmée**

**Triple validation pour H1, H2, H3 :**
1. **EDA** : Corrélations et V de Cramér
2. **Clustering** : Groupes naturels sans utiliser le target
3. **Modèle** : Coefficients de la régression logistique

> La situation professionnelle actuelle (`is_employed`, `city_development_index`) est plus prédictive que les caractéristiques démographiques (éducation, genre). Cette conclusion est robuste car confirmée par trois approches indépendantes.

---

## 7. Conclusion

### Résumé du Projet

**Objectif :** Prédire si un candidat cherche activement un nouvel emploi.

**Méthodologie :**
1. EDA rigoureuse avec hypothèses préalables (H1-H5)
2. Feature engineering ciblé (`is_employed` : V = 0.242)
3. Comparaison avec/sans FE (+1.4% à +1.8% gain)
4. Optimisation de deux familles : LR (linéaire) et HGB (boosting)
5. Validation du clustering non supervisé (patterns confirmés)

**Résultats :**
| Métrique | Valeur |
|----------|--------|
| ROC-AUC | **0.797** |
| Rappel (classe 1) | **77%** |
| Candidats détectés | **551/716** |

**Modèle final :** Logistic Regression optimisée (C=0.1, solver=liblinear)

**Facteurs clés de la recherche d'emploi :**
1. `city_development_index` (CDI faible → plus de mobilité)
2. `is_employed` (sans emploi → cherche activement)
3. `enrolled_university` (étudiant temps plein → cherche)

**Limites et perspectives :**
- Données déséquilibrées (3:1) malgré `class_weight='balanced'`
- 23% des candidats chercheurs non détectés (faux négatifs)
- Amélioration possible : features temporelles, données externes